# Quick start: Multiple Batch Predictions with kluster.ai

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/kluster-ai/klusterai-cookbook/blob/main/get-started-all-in-one-batch-api.ipynb)

Welcome to our Collab Notebook where you’ll learn how to use the [kluster.ai](https://kluster.ai/) **batch inference service to extract insights from large volumes of text data**. By the end of this guide, you'll be able to *summarize a dataset, generate keywords, translate text from English to Spanish, classify the text and extract the relevant keywords* all without writing a single line of code. Whether you’re familiar with LLMs or just getting started, this guide is designed to get you up and running in minutes.

In this guide, you'll follow just a few simple steps:

1.	**Enter your API Key:** The only required input. This allows access to kluster.ai API for generating predictions. If you don't have one, just sign up [here](https://platform.kluster.ai/signup) and you'll get one for free
2.	**Select a Dataset:** We offer three different datasets to choose from. Just select one from the provided options to get started.
3.	**Run Configurations and Predictions:** All configuration settings are already defined. After selecting the dataset, you can proceed by simply running the provided cells, which will handle all configurations and execute batch predictions for you.

That's it! No additional setup or input is required. This notebook is designed to be as hands-off as possible to ensure a smooth experience.

### Table of contents

>>[Config](#scrollTo=xU1WBQJ7Uh09)

>>[Setup](#scrollTo=650e5559-69c7-4274-8700-a7d94791e9db)

>>[Fetch the data](#scrollTo=udPtLfTaisSw)

>>[Define the tasks](#scrollTo=w-Lux3oUjfYI)

>>[Batch Predictions](#scrollTo=OyGuHllZllct)

>>>[Create the Batch File](#scrollTo=Ew-R24Ltp5EW)

>>>[Upload Batch File to kluster.ai](#scrollTo=xArKu7-sqSiR)

>>>[Check Job progress](#scrollTo=e-ujphILqepu)

>>[Get the results](#scrollTo=TkkhIG9HU0D9)



## Config

In [1]:
from getpass import getpass
# Enter you personal kluster.ai API key (make sure in advance it has no blank spaces)
api_key = getpass("Enter your kluster.ai API key: ")

Enter your kluster.ai API key: ··········


##Setup

In [2]:
%pip install OpenAI

In [3]:
import os
import urllib.request
import pandas as pd
import requests
from openai import OpenAI
import time
import json
from IPython.display import clear_output, display

pd.set_option('display.max_columns', 1000, 'display.width', 1000, 'display.max_rows',1000, 'display.max_colwidth', 500)

In [4]:
# Set up the client
client = OpenAI(
    base_url="https://api.kluster.ai/v1",
    api_key=api_key,
)

## Fetch the data

In this section, you can choose from three different datasets to work with. Simply select the URL corresponding to the dataset you want, and the notebook will automatically fetch it for you. No extra steps are needed!

In [5]:
# Choose your dataset:
# AMZ musical instrument reviews dataset:
url = "https://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Musical_Instruments_5.json.gz"

# IMDB Top 1000 sample dataset:
#url = "https://raw.githubusercontent.com/kluster-ai/klusterai-cookbook/refs/heads/main/data/imdb_top_1000.csv"

# AG News sample dataset:
#url = "https://raw.githubusercontent.com/kluster-ai/klusterai-cookbook/refs/heads/main/data/ag_news.csv"

In [6]:
def fetch_dataset(url, file_path=None):
    # Set the default file path based on the URL if none is provided
    if not file_path:
        file_path = os.path.join("data", os.path.basename(url))

    # Create the directory if it does not exist
    os.makedirs(os.path.dirname(file_path), exist_ok=True)

    # Download the file
    urllib.request.urlretrieve(url, file_path)
    print(f"Dataset downloaded and saved as {file_path}")

    # Load and process the dataset based on URL content
    if "imdb_top_1000.csv" in url:
        df = pd.read_csv(file_path)
        df['text'] = df['Series_Title'].astype(str) + ": " + df['Overview'].astype(str)
        df = df[['text']]
    elif "ag_news" in url:
        df = pd.read_csv(file_path, header=None, names=["label", "title", "description"])
        df['text'] = df['title'].astype(str) + ": " + df['description'].astype(str)
        df = df[['text']]
    elif "reviews_Musical_Instruments_5.json.gz" in url:
        df = pd.read_json(file_path, compression='gzip', lines=True)
        df.rename({'reviewText': 'text'}, axis=1, inplace=True)
        df = df[['text']]
    else:
        raise ValueError("URL does not match any known dataset format.")

    return df.tail(5).reset_index(drop=True)

df = fetch_dataset(url=url, file_path=None)
df.head()

Dataset downloaded and saved as data/reviews_Musical_Instruments_5.json.gz


,text
0,"Great, just as expected. Thank to all."
1,"I've been thinking about trying the Nanoweb strings for a while, but I was a bit put off by the high price (they cost about twice as much as the uncharted strings I've been buying) and the comments of some reviewers that the tone of coated strings is noticeably duller. I was intrigued by the promise of long life, though; I have a Taylor Big Baby that I bought used, and which came with a set of Nanowebs that had probably been on it for a year- and they didn't sound at all like old strings. T..."
2,"I have tried coated strings in the past ( including Elixirs) and have never been very fond of them. Whenever I tried them I felt a certain disconnect from my guitar. Somewhat reminiscent of wearing condom. Not that I hated them, just didn't really love them. These are the best ones I've tried so far. I still don't like them as much as regular strings but because of the type of gigs I mostly do these seem to be a reasonable trade off. If you need a longer lasting string for whatever the reaso..."
3,"Well, MADE by Elixir and DEVELOPED with Taylor Guitars ... these strings were designed for the new 800 (Rosewood) series guitars that came out this year (2014) ... the promise is a &#34;bolder high end, fuller low end&#34; ... I am a long-time Taylor owner and favor their 800 series (Rosewood/Spruce is my favorite combo in tone woods) ... I have almost always used Elixir Nanoweb Phosphor Bronze lights on my guitars ... I like not only the tone but the feel and longevity of these strings ... ..."
4,"These strings are really quite good, but I wouldn't call them perfect. The unwound strings are not quite as bright as I am accustomed to, but they still ring nicely. This is the only complaint I have about these strings. If the unwound strings were a tiny bit brighter, these would be 5-star strings. As it stands, I give them 4.5 stars... not a big knock, actually.The low-end on the wound strings is very nice and quite warm. I put these on a jumbo and it definitely accentuates the &#34;j..."


Now that you have an idea of what the original text looks like, we can move forward with defining the tasks to perform.

## Define the tasks

In this section, we’ve predefined five tasks for the model to execute based on common customer use cases:
1.	Sentiment Analysis
2.	Translation
3.	Summarization
4.	Topic Classification
5.	Keyword Extraction

If you’re happy with these tasks, you can simply run the code as-is. However, if you’d like to customize the tasks, then feel free to modify the prompts (or add new ones) to make personal requests.

In [7]:
SYSTEM_PROMPTS = {
    'sentiment': '''
    Analyze the sentiment of the given text. Provide only a JSON object with the following structure:
    {
        "sentiment": string, // "positive", "negative", or "neutral"
        "confidence": float, // A value between 0 and 1 indicating your confidence in the sentiment analysis
    }
    ''',

    'translation': '''
    Translate the given text from English to Spanish, paraphrase, rewrite or perform cultural adaptations for the text to make sense in Spanish. Provide only a JSON object with the following structure:
    {
        "translation": string, // The Spanish translation
        "notes": string // Any notes about the translation, such as cultural adaptations or challenging phrases (max 500 words). Write this mainly in english.
    }
    ''',

    'summary': '''
    Summarize the main points of the given text. Provide only a JSON object with the following structure:
    {
        "summary": string, // A concise summary of the text (max 100 words)
    }
    ''',

    'topic_classification': '''
    Classify the main topic of the given text based on the following categories: "politics", "sports", "technology", "science", "business", "entertainment", "health", "other". Provide only a JSON object with the following structure:
    {
        "category": string, // The primary category of the provided text
        "confidence": float, // A value between 0 and 1 indicating confidence in the classification
    }
    ''',

    'keyword_extraction': '''
    Extract relevant keywords from the given text. Provide only a JSON object with the following structure:
    {
        "keywords": string[], // An array of up to 5 keywords that best represent the text content
        "context": string // Briefly explain how each keyword is relevant to the text (max 200 words)
    }
    '''
}

## Batch Predictions

Now we’re diving into the main event of this notebook: running batch predictions! This is the core purpose of the tool, and it’s where the magic happens.

To execute the batch predictions, we’ll follow two simple steps:
1.	**Create the Batch File:** We’ll generate a file containing a collection of tasks (the ones defined earlier) to be processed by the model.
2.	**Upload the Batch File:** Once the batch file is ready, we’ll upload it to the kluster.ai platform using the API, where the tasks will be executed.

Everything is set up for you – just run the cells below to watch it all come together!

### Create the Batch File

This example selects the "klusterai/Meta-Llama-3.1-405B-Instruct-Turbo" model. If you'd like to use a different model feel free to change the model's name in the following cell. Please refer to our [documentation](https://docs.kluster.ai/getting-started/#list-supported-models) for a list of the models we support.

In [8]:
def create_tasks(df, task_type, system_prompt):
    tasks = []
    for index, row in df.iterrows():
        content = row['text']
        if task_type == 'script':
            content = f"Title: {row['Series_Title']}\n\nDescription: {content}"

        task = {
            "custom_id": f"{task_type}-{index}",
            "method": "POST",
            "url": "/v1/chat/completions",
            "body": {
                "model": "klusterai/Meta-Llama-3.1-405B-Instruct-Turbo",
                "temperature": 0.5,
                "messages": [
                    {"role": "system", "content": system_prompt},
                    {"role": "user", "content": content}
                ],
            }
        }
        tasks.append(task)
    return tasks

def save_tasks(tasks, task_type):
    filename = f"data/batch_tasks_{task_type}.jsonl"
    with open(filename, 'w') as file:
        for task in tasks:
            file.write(json.dumps(task) + '\n')
    return filename

In [9]:
tasks = []

for task_type, system_prompt in SYSTEM_PROMPTS.items():
    task_list = create_tasks(df, task_type, system_prompt)
    filename = save_tasks(task_list, task_type)
    tasks.append((task_type, filename))

### Upload Batch File to kluster.ai

In [10]:
def create_batch_job(file_name):
    print(f"Creating batch job for {file_name}")
    batch_file = client.files.create(
        file=open(file_name, "rb"),
        purpose="batch"
    )

    batch_job = client.batches.create(
        input_file_id=batch_file.id,
        endpoint="/v1/chat/completions",
        completion_window="24h"
    )

    return batch_job

In [11]:
batch_jobs = []

for task_type, file_name in tasks:
    job = create_batch_job(file_name)
    batch_jobs.append((f"{task_type}", job))

Creating batch job for data/batch_tasks_sentiment.jsonl
Creating batch job for data/batch_tasks_translation.jsonl
Creating batch job for data/batch_tasks_summary.jsonl
Creating batch job for data/batch_tasks_topic_classification.jsonl
Creating batch job for data/batch_tasks_keyword_extraction.jsonl


### Check Job progress

The batch jobs have been created, and all tasks are now being processed! In the following section, we’ll monitor the status of each job to see how they’re progressing. Let’s take a look and keep track of their completion.

**REMEMBER:** The time to complete a batch inference job can take up to 24 hours. Although, it typically completes in much less time.

In [12]:
def parse_json_objects(data_string):
    if isinstance(data_string, bytes):
        data_string = data_string.decode('utf-8')

    json_strings = data_string.strip().split('\n')
    json_objects = []

    for json_str in json_strings:
        try:
            json_obj = json.loads(json_str)
            json_objects.append(json_obj)
        except json.JSONDecodeError as e:
            print(f"Error parsing JSON: {e}")

    return json_objects

all_completed = False
while not all_completed:
    all_completed = True
    output_lines = []

    for i, (job_type, job) in enumerate(batch_jobs):
        updated_job = client.batches.retrieve(job.id)
        batch_jobs[i] = (job_type, updated_job)

        if updated_job.status != "Completed":
            all_completed = False
            completed = updated_job.request_counts.completed
            total = updated_job.request_counts.total
            output_lines.append(f"{job_type.capitalize()} job status: {updated_job.status} - Progress: {completed}/{total}")
        else:
            output_lines.append(f"{job_type.capitalize()} job completed!")

    # Clear the output and display updated status
    clear_output(wait=True)
    for line in output_lines:
        display(line)

    if not all_completed:
        time.sleep(10)

'Sentiment job completed!'

'Translation job completed!'

'Summary job completed!'

'Topic_classification job completed!'

'Keyword_extraction job completed!'

## Get the results

In [13]:
for job_type, job in batch_jobs:
    batch_job = client.batches.retrieve(job.id)
    result_file_id = batch_job.output_file_id
    result = client.files.content(result_file_id).content
    results = parse_json_objects(result)

    for res in results:
        task_id = res['custom_id']
        index = task_id.split('-')[-1]
        result = res['response']['body']['choices'][0]['message']['content']
        text = df.iloc[int(index)]['text']
        print(f'\n -------------------------- \n')
        print(f"Task ID: {task_id}. \n\nTEXT: {text}\n\nRESULT: {result}")


 -------------------------- 

Task ID: sentiment-0. 

TEXT: Great, just as expected.  Thank to all.

RESULT: {
        "sentiment": "positive",
        "confidence": 0.9
}

 -------------------------- 

Task ID: sentiment-1. 

TEXT: I've been thinking about trying the Nanoweb strings for a while, but I was a bit put off by the high price (they cost about twice as much as the uncharted strings I've been buying)  and the comments of some reviewers that the tone of coated strings is noticeably duller. I was intrigued by the promise of long life, though; I have a Taylor Big Baby that I bought used, and which came with a set of Nanowebs that had probably been on it for a year- and they didn't sound at all like old strings. This review set gave me a chance to finally see for myself how they sound when new.I'd just changed the strings on my 1970s Gibson Gospel a week ago, so I decided that would be my reference. The Nanowebs went on my 1970s Guild D-35. Both are well broken in, solid spruce 